In [2]:
%conda install -c conda-forge pandas
%conda install -c conda-forge scikit-learn

Solving environment...
Solving took 1.1705999999046326 seconds
  Name                          Version                       Build                         Channel                       
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
~ comm                          0.2.2 -> 0.2.3                pyhd8ed1ab_1 -> pyhe01879c_0  https://repo.prefix.dev/conda-forge -> https://repo.prefix.dev/conda-forge/
~ contourpy                     1.3.2 -> 1.3.3                py313h3a67976_0 -> py313h3a67976_1https://repo.prefix.dev/emscripten-forge-dev -> https://repo.prefix.dev/emscripten-forge-dev/
~ emscripten-abi                3.1.73 -> 3.1.73              h267e887_10 -> h267e887_12    https://repo.prefix.dev/emscripten-forge-dev -> https://repo.prefix.dev/emscripten-forge-dev/
~ fonttools                     4.39.4 -> 4.39.4              h7223423_0 -> py313h6e18b15_1 https://repo.prefix.dev/emscripten-forge-dev -> https://r

In [4]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd 
import sklearn

data = pd.read_csv("preprocessed_TEST_data3.csv")
data



,SibSp,Parch,Fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,embarked_U,Pclass_1,Pclass_2,Pclass_3,"Cat_Age_(0, 10]","Cat_Age_(10, 20]","Cat_Age_(20, 30]","Cat_Age_(30, 40]","Cat_Age_(40, 50]","Cat_Age_(50, 60]","Cat_Age_(60, 70]","Cat_Age_(70, 80]"
0,0,0,7.8292,False,True,False,True,False,0,False,False,True,False,False,False,True,False,False,False,False
1,1,0,7.0000,True,False,False,False,True,0,False,False,True,False,False,False,False,True,False,False,False
2,0,0,9.6875,False,True,False,True,False,0,False,True,False,False,False,False,False,False,False,True,False
3,0,0,8.6625,False,True,False,False,True,0,False,False,True,False,False,True,False,False,False,False,False
4,1,1,12.2875,True,False,False,False,True,0,False,False,True,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0,0,8.0500,False,True,False,False,True,0,False,False,True,False,False,False,True,False,False,False,False
414,0,0,108.9000,True,False,True,False,False,0,True,False,False,False,False,False,True,False,False,False,False
415,0,0,7.2500,False,True,False,False,True,0,False,False,True,False,False,False,True,False,False,False,False
416,0,0,8.0500,False,True,False,False,True,0,False,False,True,False,False,False,True,False,False,False,False


In [5]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd 
import sklearn
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score 
from sklearn.model_selection import train_test_split


data = pd.read_csv("./preprocessed_data5.csv") 

features = data.drop(["Survived"], axis=1)
labels = data["Survived"] 

features_train, features_v_t, labels_train, labels_v_t = train_test_split(features, labels, test_size=0.4)
features_validation, features_test, labels_validation, labels_test = train_test_split(features_v_t, 
                                                                                      labels_v_t, 
                                                                                      test_size=0.5)





In [6]:
rf_model = RandomForestClassifier()
rf_model.fit(features_train, labels_train)

gb_model = GradientBoostingClassifier()
gb_model.fit(features_train, labels_train)

ab_model = AdaBoostClassifier()
ab_model.fit(features_train, labels_train)

rf_model.score(features_train, labels_train)

0.951310861423221

In [7]:
gb_model.score(features_train, labels_train)

0.900749063670412

In [8]:
ab_model.score(features_train, labels_train)

0.8108614232209738

In [9]:
from sklearn.model_selection import GridSearchCV

rf_model_improved = RandomForestClassifier(random_state=0)

param_grid = { 
    'n_estimators': [50, 100, 200, 300], 
    'max_depth': [2, 3, 4, 5, 10], 
    'min_samples_split': [2, 5, 10], 
    'max_features': ['sqrt', 'log2'] 
}

grid = GridSearchCV(rf_model, param_grid, cv=5, scoring='accuracy', n_jobs=-1) 
grid.fit(features_train, labels_train) 

grid.best_params_

{'max_depth': 5,
 'max_features': 'log2',
 'min_samples_split': 10,
 'n_estimators': 50}

In [10]:
grid.best_score_

np.float64(0.823928760359725)

In [12]:
new_rf_model = grid.best_estimator_
new_rf_model.score(features_validation, labels_validation)

0.797752808988764

In [13]:
new_rf_model.score(features_test, labels_test)

0.8324022346368715

In [15]:
test_data = pd.read_csv("./preprocessed_TEST_data3.csv", index_col=None) 

test_data_raw = pd.read_csv("./test_data.csv", index_col=None)
testLabels = test_data_raw["PassengerId"]

rf_Result = np.array(new_rf_model.predict(test_data))

passenger_IDs = np.arange(892, 892 + len(gb_model.predict(test_data)))

df = pd.DataFrame({
    'PassengerId': passenger_IDs, 
    'Survived': rf_Result
}) 

df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [17]:
df.to_csv("result2.csv", index=None)